In [32]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob


In [33]:
# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    block_files = np.array(data['filenames'])
    block_targets = np_utils.to_categorical(np.array(data['target']), 6)
    return block_files, block_targets

In [34]:
# load train, test, and validation datasets
train_files, train_targets = load_dataset('block_image/train')
valid_files, valid_targets = load_dataset('block_image/valid')
test_files, test_targets = load_dataset('block_image/test')

In [35]:
# load list of dog names
block_names = [item[20:-1] for item in sorted(glob("block_image/train/*/"))]

# print statistics about the dataset
print('There are %d total block categories.' % len(block_names))
print('There are %s total block images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training block images.' % len(train_files))
print('There are %d validation block images.' % len(valid_files))
print('There are %d test block images.'% len(test_files))

There are 6 total block categories.
There are 1080 total block images.

There are 989 training block images.
There are 61 validation block images.
There are 30 test block images.


In [36]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [37]:
train_files[:10]

array(['block_image/train/000.none/block_010912462113.jpg',
       'block_image/train/000.none/block_010912462615.jpg',
       'block_image/train/000.none/block_0109124517.jpg',
       'block_image/train/000.none/block_01091246264.jpg',
       'block_image/train/000.none/block_010912474417.jpg',
       'block_image/train/005.gray/block_01091246160.jpg',
       'block_image/train/001.red/block_010912473118.jpg',
       'block_image/train/003.blue/block_010912474115.jpg',
       'block_image/train/000.none/block_01091246272.jpg',
       'block_image/train/003.blue/block_01091245211.jpg'], dtype='<U51')

In [38]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255



100%|██████████| 30/30 [00:00<00:00, 586.57it/s]


In [39]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()


model.add( Conv2D(16,  kernel_size = (2, 2),  input_shape=(224, 224, 3), activation = 'relu', strides = 1, padding = 'valid'))
model.add( MaxPooling2D(pool_size = (2, 2),  strides = 2))
model.add( Conv2D(32, kernel_size = (2, 2),  activation = 'relu', strides = 1, padding = 'valid'))
model.add( MaxPooling2D(pool_size = (2, 2),  strides = 2))
model.add( Conv2D(64, kernel_size = (2, 2),  activation = 'relu', strides = 1, padding = 'valid'))
model.add( MaxPooling2D(pool_size = (2, 2),  strides = 2))
model.add( GlobalAveragePooling2D())
model.add( Dense(units = 6, activation = 'softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 223, 223, 16)      208       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 110, 110, 32)      2080      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 55, 55, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 54, 54, 64)        8256      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 27, 27, 64)        0         
_________________________________________________________________
global_average_pooling2d_3 ( (None, 64)                0         
__________

In [40]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [41]:
from keras.callbacks import ModelCheckpoint  

epochs = 5


checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 989 samples, validate on 61 samples
Epoch 1/5
989/989 [==============================] - 33s 33ms/step - loss: 0.9542 - acc: 0.7270 - val_loss: 2.2133 - val_acc: 0.1803

Epoch 00001: val_loss improved from inf to 2.21335, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 2/5
989/989 [==============================] - 30s 30ms/step - loss: 0.8647 - acc: 0.7361 - val_loss: 2.5010 - val_acc: 0.1803

Epoch 00002: val_loss did not improve from 2.21335
Epoch 3/5
989/989 [==============================] - 25s 25ms/step - loss: 0.8058 - acc: 0.7371 - val_loss: 2.3672 - val_acc: 0.1803

Epoch 00003: val_loss did not improve from 2.21335
Epoch 4/5
989/989 [==============================] - 24s 25ms/step - loss: 0.7287 - acc: 0.7674 - val_loss: 1.9483 - val_acc: 0.2951

Epoch 00004: val_loss improved from 2.21335 to 1.94827, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 5/5
989/989 [==============================] - 25s 25ms/step - loss: 0.6610 - acc: 

In [42]:

model.load_weights('saved_models/weights.best.from_scratch.hdf5')

In [72]:
data = load_files('block_image/input')
input_files = np.array(data['filenames'])
input_tensors = paths_to_tensor(input_files).astype('float32')/255



  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 501.03it/s]

In [73]:
#input_tensors
model.predict_proba(input_tensors, batch_size = 2)



array([[0.30650052, 0.16180253, 0.17130686, 0.20741056, 0.08353481,
        0.0694447 ],
       [0.05742539, 0.7942814 , 0.0434706 , 0.01336617, 0.074697  ,
        0.01675936],
       [0.17309605, 0.68421966, 0.04351623, 0.02496198, 0.05300141,
        0.0212047 ]], dtype=float32)

In [21]:
# get index of predicted dog breed for each image in test set
block_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(block_predictions)==np.argmax(test_targets, axis=1))/len(block_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

[1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 3, 0, 0, 1, 1, 0, 0, 1, 0, 0]
Test accuracy: 30.0000%
